In [32]:
from imutils.perspective import four_point_transform
from skimage.filters import threshold_local
import numpy as np
import cv2 as cv
import imutils
import os

In [33]:
ARTIFACT_PATH = os.path.join(os.path.abspath(".."),"artifacts")
models_path = os.path.join(ARTIFACT_PATH,"models")
results_path = os.path.join(ARTIFACT_PATH,"results","License_Plate")
samples_path = os.path.join(ARTIFACT_PATH,"samples")

In [254]:
image = cv.imread(os.path.join(results_path,"plate0.jpg"))
ratio = image.shape[0]/image.shape[0]
orig = image.copy()
# image = imutils.resize(image, height=300)

gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
gray = cv.GaussianBlur(gray, (7,7), 0)
edged = cv.Canny(gray, 150, 255)

print("STEP 1: Edge Detection")
cv.imshow("Image", image)
cv.imshow("Edged", edged)

cv.waitKey(0)
cv.destroyAllWindows()

STEP 1: Edge Detection


In [255]:
cnts = cv.findContours(edged.copy(), cv.RETR_LIST, cv.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)
cnts = sorted(cnts, key = cv.contourArea, reverse = True)[:5]
# loop over the contours
for c in cnts:
	# approximate the contour
	peri = cv.arcLength(c, True)
	approx = cv.approxPolyDP(c, 0.02 * peri, True)
	# if our approximated contour has four points, then we
	# can assume that we have found our screen
	if len(approx) == 4:
		screenCnt = approx
		break
# show the contour (outline) of the piece of paper
print("STEP 2: Find contours of paper")
cv.drawContours(image, [screenCnt], -1, (0, 255, 0), 1)
cv.imshow("Outline", image)
cv.waitKey(0)
cv.destroyAllWindows()

STEP 2: Find contours of paper


In [260]:
# apply the four point transform to obtain a top-down
# view of the original image
warped = four_point_transform(orig, screenCnt.reshape(4, 2) * ratio)
# convert the warped image to grayscale, then threshold it
# to give it that 'black and white' paper effect
warped = cv.cvtColor(warped, cv.COLOR_BGR2GRAY)
# T = threshold_local(warped, 11, offset = 10, method = "gaussian")
# warped = (warped > T).astype("uint8") * 255
# show the original and scanned images
print("STEP 3: Apply perspective transform")
cv.imshow("Original", orig)
cv.imshow("Scanned", warped)
cv.waitKey(0)
cv.destroyAllWindows()

# save_path = os.path.join(ARTIFACT_PATH,"results","cropped")



STEP 3: Apply perspective transform


In [261]:
# Apply histogram equalization
equalized_image = cv.equalizeHist(warped)

print("STEP 4: Apply Histogram equalization")
cv.imshow("Scanned", equalized_image)
cv.waitKey(0)
cv.destroyAllWindows()

STEP 4: Apply Histogram equalization


In [263]:
height, width = equalized_image.shape[:2]
clip_rows = int(0.25 * height)

# Clip the top and bottom rows
clipped_image = equalized_image[clip_rows:-clip_rows, :]
clipped_image_no_hist = warped[clip_rows:-clip_rows, :]

# Display or save the clipped image
cv.imshow('Clipped Image', clipped_image)
# cv.imshow('Clipped Image', clipped_image)
cv.imshow('Clipped Image no hist', clipped_image_no_hist)
cv.waitKey(0)
cv.destroyAllWindows()

cv.imwrite(os.path.join(ARTIFACT_PATH,"results", "clipped.jpg"),clipped_image)

True

In [205]:
clipped_image = imutils.resize(clipped_image, height = 50)
clipped_image.shape

(50, 155)

In [281]:
blurred = cv.GaussianBlur(clipped_image, (9,9), 0)
thresh = cv.adaptiveThreshold(clipped_image, 255, cv.ADAPTIVE_THRESH_MEAN_C, cv.THRESH_BINARY_INV, 29, 55)

threshold_value = 90
max_value = 255
# ret, thresh = cv.threshold(blurred, threshold_value, max_value, cv.THRESH_BINARY_INV)

cv.imshow("test", imutils.resize(thresh, height = 50))
cv.waitKey(0)
cv.destroyAllWindows()

In [282]:
_, labels = cv.connectedComponents(thresh)
mask = np.zeros(thresh.shape, dtype="uint8")

In [300]:
total_pixels = clipped_image.shape[0] * clipped_image.shape[1]
lower = total_pixels // 50
upper = total_pixels // 10
print(total_pixels)

901


In [301]:
lower

18

In [302]:
upper

90

In [303]:
np.unique(labels)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14],
      dtype=int32)

In [306]:
for(i, label) in enumerate(np.unique(labels)):

  if label == 0:
    continue

  labelMask = np.zeros(thresh.shape, dtype="uint8")
  labelMask[labels == label] = 255
  numPixels = cv.countNonZero(labelMask)


  if numPixels > lower and numPixels < upper :
    mask = cv.add(mask, labelMask)

cv.imshow("test", imutils.resize(mask, height = 50))
cv.waitKey(0)
cv.destroyAllWindows()

In [307]:
cnts,_ = cv.findContours(mask.copy(), cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
boundingBoxes = [cv.boundingRect (c) for c in cnts]

In [308]:
boundingBoxes

[(34, 12, 2, 3),
 (30, 3, 3, 14),
 (23, 3, 5, 13),
 (14, 3, 9, 13),
 (8, 3, 5, 13),
 (2, 3, 5, 13),
 (46, 2, 6, 13),
 (41, 2, 5, 13),
 (35, 2, 5, 13),
 (38, 0, 4, 1)]

In [309]:
import functools

def compare(rect1, rect2):
  if abs(rect1[1] - rect2[1]) > 10:
    return rect1[1] - rect2[1]
  else:
    return rect1[0] - rect2[0]
boundingBoxes = sorted(boundingBoxes, key=functools.cmp_to_key(compare))

In [319]:
for bbox in boundingBoxes:
    x, y, w, h = bbox
    cv.rectangle(clipped_image, (x, y), (x + w, y + h), (0, 255, 0), 1)

cv.imshow("test", imutils.resize(clipped_image, height = 50))
cv.waitKey(0)
cv.destroyAllWindows()

In [313]:
bb = boundingBoxes[0]
bb

(2, 3, 5, 13)

In [315]:
bb[0]+bb[2]

7

In [318]:
crop = clipped_image[bb[1]:(bb[1]+bb[3]),bb[0]:(bb[0]+bb[2])]

cv.imshow("test", imutils.resize(crop, height = 50))
cv.waitKey(0)
cv.destroyAllWindows()